In [29]:
%matplotlib inline
from pyquil import Program, get_qc
from pyquil.gates import *
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
from IPython.display import clear_output
import collections

In [30]:
def live_plot(data_dict, figsize=(7,5), title='Ramsey'):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('Wait time')
    plt.legend(loc='upper left') # the plot evolves to the right
    plt.show()

In [31]:
def ramsey(qubit, wait_time, frame, frame_freq, shots=10_000):

    p = Program(
        "DECLARE ro BIT",
        f'SET-FREQUENCY {frame} {frame_freq}',
        f"RESET",
        f"RX(pi/2) {qubit}",
        f"DELAY {qubit} {wait_time}e-9",
        f"RX(pi/2) {qubit}",
        f"MEASURE {qubit} ro"
    ).wrap_in_numshots_loop(shots)
        
    return p

In [32]:
def get_cals(qubit, qc):
    cals = qc.compiler.get_calibration_program()
    rf_frames = list(filter(lambda f: f[0].name == "rf", cals.frames.items()))

    # save qubit frame and frequency
    (frame, frame_def) = rf_frames[qubit]
    frequency = frame_def.initial_frequency
    center = frame_def.center_frequency
    print('qubit frame: ', frame, '\tfrequency: ', frequency, '\tcenter', center)
    
    return frame, frequency, center

In [34]:
# runtime parameters
qubit = 7
shots = 10_000

q_computer = 'Aspen-11'
qc = get_qc(q_computer)

frame, freq, center = get_cals(qubit, qc)
frame_freq = center - 400_000_000

# live logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
logging.basicConfig(filename='logs/ramsey.log', 
                    format='%(asctime)s%(msecs)03d - %(message)s', 
                    level=logging.INFO, 
                    datefmt='%d-%b-%y %H:%M:%S.')
logging.info('------------------------------------------------------------------------------   Begin')
logging.info(f'Qubit: {qubit}')
logging.info(f'Qubit frequency: {freq}')
logging.info(f'Frame frequency: {frame_freq}')
logging.info(f'Qubit center frequency: {center}')
logging.info(f'Frame: {frame}')
logging.info(f'Shots: {shots}')
logging.info(f'Quantum computer: {q_computer}')
             
# initialize loop
times = np.linspace(0,100,50)
results = collections.defaultdict(list)

for wait_time in times:

    # get quil program and compile it (log compilation time)
    prog = ramsey(qubit, wait_time, frame, frame_freq, shots)
    start_compile = time.time()
    exe = qc.compiler.native_quil_to_executable(prog)
    total_compile = time.time() - start_compile
    logging.info(f'Program compiled in {total_compile:.3f} s')
    
    # run the program (and time it)
    start_run = time.time()
    #result = qc.run(exe)
    # print(result)
    result = qc.run(exe).readout_data.get('ro')
    total_run = time.time() - start_run
    
    # update plot and log
    probability = np.sum(result) / shots
    results['Probability'].append(probability)
    live_plot(results)
    logging.info(f' wait time: {wait_time} \t-- probability: {probability} \t--- runtime: {total_run:.3f} seconds')
    
logging.info('Done\n')

qubit frame:  7 "rf" 	frequency:  3809542155.02938 	center 3750000000.0


QCSHTTPStatusError: QCS API call POST https://api.qcs.rigetti.com/v1/engagements failed with status 400: {"code":"bad_request","message":"A reservation is required to engage, but you do not have a current reservation","requestId":"3a720906-3ab0-4d58-a43a-ba7089e54812"}